In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sn
import math
import cv2
import os
import PIL

from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_csv('train.csv')
df_train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
423,424,0,3,"Danbom, Mrs. Ernst Gilbert (Anna Sigrid Maria ...",female,28.00,1,1,347080,14.4000,NaN,S
747,748,1,2,"Sinkkonen, Miss. Anna",female,30.00,0,0,250648,13.0000,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.00,0,2,347742,11.1333,NaN,S
578,579,0,3,"Caram, Mrs. Joseph (Maria Elias)",female,NaN,1,0,2689,14.4583,NaN,C
837,838,0,3,"Sirota, Mr. Maurice",male,NaN,0,0,392092,8.0500,NaN,S
36,37,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C
469,470,1,3,"Baclini, Miss. Helene Barbara",female,0.75,2,1,2666,19.2583,NaN,C
223,224,0,3,"Nenkoff, Mr. Christo",male,NaN,0,0,349234,7.8958,NaN,S
755,756,1,2,"Hamalainen, Master. Viljo",male,0.67,1,1,250649,14.5000,NaN,S
512,513,1,1,"McGough, Mr. James Robert",male,36.00,0,0,PC 17473,26.2875,E25,S


In [3]:
df_test = pd.read_csv('test.csv')

In [4]:
df_outex = pd.read_csv('gender_submission.csv')
df_outex.sample(10)

,PassengerId,Survived
70,962,1
115,1007,0
155,1047,0
305,1197,1
148,1040,0
302,1194,0
343,1235,1
365,1257,1
98,990,1
269,1161,0


In [5]:
df_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [6]:
df_test_copy = df_test.copy()

In [7]:
df_train_new = df_train.drop('Pclass',axis='columns')
df_train_new = df_train_new.drop('Name',axis='columns')
df_train_new = df_train_new.drop('SibSp',axis='columns')
df_train_new = df_train_new.drop('Parch',axis='columns')
df_train_new = df_train_new.drop('Fare',axis='columns')
df_train_new = df_train_new.drop('Embarked',axis='columns')
df_train_new = df_train_new.drop('Ticket',axis='columns')
df_train_new = df_train_new.drop('PassengerId',axis='columns')
df_train_new = df_train_new.drop('Cabin',axis='columns')
df_train_new.sample(10)

,Survived,Sex,Age
618,1,female,4.0
322,1,female,30.0
504,1,female,16.0
570,1,male,62.0
454,0,male,NaN
861,0,male,21.0
292,0,male,36.0
468,0,male,NaN
703,0,male,25.0
552,0,male,NaN


In [8]:
df_train_new['Sex'] = df_train_new['Sex'].map({'male': 1, 'female': 0})
df_train_new.sample(10)

,Survived,Sex,Age
722,0,1,34.0
208,1,0,16.0
157,0,1,30.0
150,0,1,51.0
126,0,1,NaN
742,1,0,21.0
508,0,1,28.0
500,0,1,17.0
400,1,1,39.0
316,1,0,24.0


In [9]:
df_train_new['Age'] = df_train_new['Age'].fillna(0)

In [10]:
cols_to_scale = ['Age']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train_new[cols_to_scale] = scaler.fit_transform(df_train_new[cols_to_scale])

df_train_new.sample(15)

,Survived,Sex,Age
768,0,1,0.0000
408,0,1,0.2625
463,0,1,0.6000
397,0,1,0.5750
39,1,0,0.1750
516,1,0,0.4250
830,1,0,0.1875
167,0,0,0.5625
481,0,1,0.0000
425,0,1,0.0000


In [11]:
x_train = df_train_new.drop('Survived',axis='columns')
y_train = df_train_new['Survived']

In [12]:
x_train.shape

(891, 2)

In [13]:
df_test_new = df_test.drop('Pclass',axis='columns')
df_test_new = df_test_new.drop('Name',axis='columns')
df_test_new = df_test_new.drop('SibSp',axis='columns')
df_test_new = df_test_new.drop('Parch',axis='columns')
df_test_new = df_test_new.drop('Fare',axis='columns')
df_test_new = df_test_new.drop('Embarked',axis='columns')
df_test_new = df_test_new.drop('Ticket',axis='columns')
df_test_new = df_test_new.drop('PassengerId',axis='columns')
df_test_new = df_test_new.drop('Cabin',axis='columns')
df_test_new['Sex'] = df_test_new['Sex'].map({'male': 1, 'female': 0})
df_test_new['Age'] = df_test_new['Age'].fillna(0)
cols_to_scale = ['Age']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_test_new[cols_to_scale] = scaler.fit_transform(df_test_new[cols_to_scale])

In [14]:
df_test_new.shape

(418, 2)

In [15]:
df_test_new.dtypes

Sex      int64
Age    float64
dtype: object

In [16]:
def ANN(x_train, y_train, x_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(100,input_shape=(2,), activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid') #SIGMOID FOR BINARY CLASSIFICATION AND SOFTMAX FOR MULTICLASS CLASSIFICATION
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(x_train, y_train, epochs=100)
    else:
        model.fit(x_train, y_train, epochs=100, class_weight = weights)
    
    y_preds = model.predict(x_test)
    y_preds = np.round(y_preds)
    
    return y_preds

In [17]:
y_preds = ANN(x_train, y_train, df_test_new, 'binary_crossentropy', -1)

Epoch 1/100
28/28 [==============================] - 1s 2ms/step - loss: 0.6356 - accuracy: 0.5657
Epoch 2/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5743 - accuracy: 0.6431
Epoch 3/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5523 - accuracy: 0.7823
Epoch 4/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5416 - accuracy: 0.7868
Epoch 5/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5296 - accuracy: 0.7868
Epoch 6/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5220 - accuracy: 0.7868
Epoch 7/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5184 - accuracy: 0.7868
Epoch 8/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5154 - accuracy: 0.7868
Epoch 9/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5153 - accuracy: 0.7868
Epoch 10/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5142 - accuracy: 0.7868
Epoch 11/

28/28 [==============================] - 0s 3ms/step - loss: 0.5134 - accuracy: 0.7868
Epoch 84/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5131 - accuracy: 0.7868
Epoch 85/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5131 - accuracy: 0.7868
Epoch 86/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5130 - accuracy: 0.7868
Epoch 87/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5121 - accuracy: 0.7868
Epoch 88/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5132 - accuracy: 0.7868
Epoch 89/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5134 - accuracy: 0.7868
Epoch 90/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5139 - accuracy: 0.7868
Epoch 91/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5151 - accuracy: 0.7868
Epoch 92/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5122 - accuracy: 0.7868
Epoch 93/100


In [18]:
y_preds_flattened = y_preds.flatten()

In [19]:
new_column_series = pd.Series(y_preds_flattened)
df_test_copy['Survived'] = new_column_series
df_test_copy.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
237,1129,3,"Baccos, Mr. Raffull",male,20.00,0,0,2679,7.2250,NaN,C,0.0
348,1240,2,"Giles, Mr. Ralph",male,24.00,0,0,248726,13.5000,NaN,S,0.0
134,1026,3,"Dintcheff, Mr. Valtcho",male,43.00,0,0,349226,7.8958,NaN,S,0.0
81,973,1,"Straus, Mr. Isidor",male,67.00,1,0,PC 17483,221.7792,C55 C57,S,0.0
229,1121,2,"Hocking, Mr. Samuel James Metcalfe",male,36.00,0,0,242963,13.0000,NaN,S,0.0
80,972,3,"Boulos, Master. Akar",male,6.00,1,1,2678,15.2458,NaN,C,0.0
142,1034,1,"Ryerson, Mr. Arthur Larned",male,61.00,1,3,PC 17608,262.3750,B57 B59 B63 B66,C,0.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.6875,NaN,Q,0.0
201,1093,3,"Danbom, Master. Gilbert Sigvard Emanuel",male,0.33,0,2,347080,14.4000,NaN,S,0.0
375,1267,1,"Bowen, Miss. Grace Scott",female,45.00,0,0,PC 17608,262.3750,NaN,C,1.0


In [20]:
df_test_copy_new = df_test_copy.drop('Pclass',axis='columns')
df_test_copy_new = df_test_copy_new.drop('Name',axis='columns')
df_test_copy_new = df_test_copy_new.drop('SibSp',axis='columns')
df_test_copy_new = df_test_copy_new.drop('Parch',axis='columns')
df_test_copy_new = df_test_copy_new.drop('Fare',axis='columns')
df_test_copy_new = df_test_copy_new.drop('Embarked',axis='columns')
df_test_copy_new = df_test_copy_new.drop('Ticket',axis='columns')
df_test_copy_new = df_test_copy_new.drop('Age',axis='columns')
df_test_copy_new = df_test_copy_new.drop('Cabin',axis='columns')
df_test_copy_new = df_test_copy_new.drop('Sex',axis='columns')

df_test_copy_new.sample(10)

,PassengerId,Survived
305,1197,1.0
133,1025,0.0
40,932,0.0
100,992,1.0
271,1163,0.0
348,1240,0.0
247,1139,0.0
187,1079,0.0
34,926,0.0
249,1141,1.0


In [21]:
df_test = df_test_copy_new
df_test.shape

(418, 2)

In [22]:
df_test.to_csv('Titanic_Survival_Prediction_Kaggle.csv', index=False)